# 原資産価格の算出

- 日経225オプションは日経平均株価が原資産価格
- J-Quants APIでは日経225指数の当日の終値が入力されている [^1]
- 日経225オプションの場合、当限月の先物（mini）の価格を使うのが望ましい
    - 指数そのままでは、配当落ちが考慮されていない
- J-Quants APIでは先物のデータが提供されていない（2023年10月時点）ので、オプションの価格から原資産価格を算出

[^1]: [J-Quants API Quick Startガイド](https://github.com/J-Quants/event/blob/main/20230921/20230921_jquants_option_data.ipynb)



## プットコールパリティ

$$C(t) - P(t) = S(t)- K \cdot e^{-r(T-t)}$$

- $C(t)$ : コール価格
- $P(t)$ : プット価格
- $S(t)$ : 原資産価格
- $K$ : 権利行使価格
- $r$ : 無リスク金利
- $T-t$ : 満期までの期間

> https://en.wikipedia.org/wiki/Put%E2%80%93call_parity

J-Quants APIから提供されている原資産価格を確認

In [1]:
import jquants_derivatives

date = "2023-02-02"
cli = jquants_derivatives.Client()
df = cli.get_option_index_option(date)
groupby_contract_month = df.groupby("ContractMonth")
groupby_contract_month["UnderlyingPrice"].unique()

ContractMonth
2023-02    [27402.05]
2023-03    [27402.05]
2023-04    [27402.05]
2023-05    [27402.05]
2023-06    [27402.05]
2023-07    [27402.05]
2023-08    [27402.05]
2023-09    [27402.05]
2023-10    [27402.05]
2023-12    [27402.05]
2024-03    [27402.05]
2024-06    [27402.05]
2024-12    [27402.05]
2025-06    [27402.05]
2025-12    [27402.05]
2026-06    [27402.05]
2026-12    [27402.05]
2027-06    [27402.05]
2027-12    [27402.05]
2028-06    [27402.05]
2028-12    [27402.05]
2029-06    [27402.05]
2029-12    [27402.05]
2030-06    [27402.05]
2030-12    [27402.05]
Name: UnderlyingPrice, dtype: object

In [2]:
# 2023-02のDataFrame
df_202302 = (
    df.groupby("ContractMonth")
    .get_group("2023-02")
    .set_index("StrikePrice", drop=False)
)
# 2023-03のDataFrame
df_202303 = (
    df.groupby("ContractMonth")
    .get_group("2023-03")
    .set_index("StrikePrice", drop=False)
)
# プットとコールにDataFrameを分割
groupby_put_call_div_202302 = df_202302.groupby("PutCallDivision")
groupby_put_call_div_202303 = df_202303.groupby("PutCallDivision")
put_202302 = groupby_put_call_div_202302.get_group(1)
call_202302 = groupby_put_call_div_202302.get_group(2)
put_202303 = groupby_put_call_div_202303.get_group(1)
call_202303 = groupby_put_call_div_202303.get_group(2)
s_202302 = df_202302.loc[:, "UnderlyingPrice"].iloc[0]
s_202303 = df_202303.loc[:, "UnderlyingPrice"].iloc[0]

J-Quants APIから提供されている原資産価格

In [3]:
print(f"{s_202302=}, {s_202303=}")

s_202302=27402.05, s_202303=27402.05


### 無リスク金利

[J-Quants API Quick Startガイド](https://github.com/J-Quants/event/blob/main/20230921/20230921_jquants_option_data.ipynb) より

> 理論価格計算用金利（Interest Rate）  
> [参考URL](https://www.jpx.co.jp/ose-toshijuku/column/futures_option/option/03.html)
> 
> 理論価格を計算する際に使用している金利であり、全銀協の日本円TIBORレートを使用しています。  
> それぞれの限月に応じ、３か月や６か月金利等を使用しています。  

In [4]:
# 2023-02、2023-03の無リスク金利
r_202302 = df_202302.loc[:, "InterestRate"].unique()[0] * 0.01
r_202303 = df_202303.loc[:, "InterestRate"].unique()[0] * 0.01
print(f"{r_202302=}, {r_202303=}")

r_202302=-0.000391, r_202303=0.000518


### 満期までの期間

- SQ日（LastTradingDay） $T$ から当日 $t$ を引いて、年に変換
- ここでは、1年を365日とする
    - 休日を除いて250日などにする場合もある

In [5]:
import pandas as pd

YEAR_TO_SECONDS = 31_536_000
t_202302 = (
    df_202302.loc[:, "LastTradingDay"].iloc[0] - pd.Timestamp(date)
).total_seconds() / YEAR_TO_SECONDS
t_202303 = (
    df_202303.loc[:, "LastTradingDay"].iloc[0] - pd.Timestamp(date)
).total_seconds() / YEAR_TO_SECONDS
print(f"{t_202302=}, {t_202303=}")

t_202302=0.019178082191780823, t_202303=0.0958904109589041


### ATM

In [6]:
atm_202302 = abs(df_202302.loc[:, "StrikePrice"] - s_202302).idxmin()
atm_202303 = abs(df_202303.loc[:, "StrikePrice"] - s_202303).idxmin()
print(f"{s_202302=}, {s_202303=}")
print(f"{atm_202302=}, {atm_202303=}")

s_202302=27402.05, s_202303=27402.05
atm_202302=27375.0, atm_202303=27375.0


ATMのプット価格、コール価格

理論上ではどの権利行使価格からも求められるが、ATMから離れた権利行使価格はITM側の流動性がなくなり、価格を得にくい

In [7]:
atm_put_202302 = put_202302.loc[atm_202302, "WholeDayClose"]
atm_call_202302 = call_202302.loc[atm_202302, "WholeDayClose"]
atm_put_202303 = put_202303.loc[atm_202303, "WholeDayClose"]
atm_call_202303 = call_202303.loc[atm_202303, "WholeDayClose"]
print(f"{atm_put_202302=}, {atm_call_202302=}, {atm_put_202303=}, {atm_call_202303=}")

atm_put_202302=225.0, atm_call_202302=270.0, atm_put_202303=500.0, atm_call_202303=515.0


$$C(t) - P(t) = S(t) - K \cdot e^{-r(T-t)}$$

から $S(t)$ を求める

$$S(t) = C(t) - P(t) + K \cdot e^{-r(T-t)}$$

In [9]:
import numpy as np

calculated_s_202302 = (
    atm_call_202302 - atm_put_202302 + atm_202302 * np.exp(-r_202302 * t_202302)
)
calculated_s_202303 = (
    atm_call_202303 - atm_put_202303 + atm_202303 * np.exp(-r_202303 * t_202303)
)
print(f"{s_202302=}, {calculated_s_202302=}")
print(f"{s_202303=}, {calculated_s_202303=}")

s_202302=27402.05, calculated_s_202302=27420.205275769644
s_202303=27402.05, calculated_s_202303=27388.64028376967


- jquants-derivativesではJ-Quants APIから提供している原資産価格をそのまま利用
- 将来的に先物価格が提供されれた場合は、置き換える可能性あり

In [10]:
from jquants_derivatives import Option

option = Option(df)
option.underlying_price

{'2023-02': 27402.05, '2023-03': 27402.05}